In [7]:
import csv
import glob
import pickle
import pandas as pd
import networkx as nx
import dask.dataframe as dd
import multiprocessing as mp
import xml.etree.ElementTree as ET

* <s> How large a group of co-authors does the average publication have? </s>
* Do authors mostly publish using always the same group of authors?
* Do authors mainly reference papers with other authors with whom they've co-authored papers (including themselves)?
* What is the distribution in time for citations of papers in general, and for papers with the highest number of citations? Do they differ?
* Is there a correlation between citations and the number of keywords that papers share? I.e. papers which share the same subject cite each other more often.
* For the most-cited papers (define your own cutoff), is the correlation in shared keywords between them and the papers that cite them different from (5) 

# Reading and combining pickles.

In [2]:
pickle_path = '/students/2021-2022/master/Rients_DSLS/pickle_6/'
pickles = glob.glob(pickle_path + '*.pkl')

In [4]:
ddf2 = dd
ddf_list = []
for pickle in pickles:
    
    df = pd.read_pickle(pickle)
    ddf = dd.from_pandas(df, npartitions=8)
    ddf_list.append(ddf)   

    print('list size: ', len(ddf_list), '/1062')

full_frame = dd.multi.concat(ddf_list)

list size:  1 /1062
list size:  2 /1062
list size:  3 /1062
list size:  4 /1062
list size:  5 /1062
list size:  6 /1062
list size:  7 /1062
list size:  8 /1062
list size:  9 /1062
list size:  10 /1062
list size:  11 /1062
list size:  12 /1062
list size:  13 /1062
list size:  14 /1062
list size:  15 /1062
list size:  16 /1062
list size:  17 /1062
list size:  18 /1062
list size:  19 /1062
list size:  20 /1062
list size:  21 /1062
list size:  22 /1062
list size:  23 /1062
list size:  24 /1062
list size:  25 /1062
list size:  26 /1062
list size:  27 /1062
list size:  28 /1062
list size:  29 /1062
list size:  30 /1062
list size:  31 /1062
list size:  32 /1062
list size:  33 /1062
list size:  34 /1062
list size:  35 /1062
list size:  36 /1062
list size:  37 /1062
list size:  38 /1062
list size:  39 /1062
list size:  40 /1062
list size:  41 /1062
list size:  42 /1062
list size:  43 /1062
list size:  44 /1062
list size:  45 /1062
list size:  46 /1062
list size:  47 /1062
list size:  48 /1062
l

In [ ]:
# Question1.
Full_df = full_frame
total = len(Full_df) 
uniques = Full_df[0].nunique().compute()
average_authors = total / uniques

In [8]:
# Writing the answers.
with open('answers.csv', mode='w') as csv_file:
    answer_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    answer_writer.writerow(['How large a group of co-authors does the average publication have?', round(average_authors,2)])
    print('The papers have', average_authors, 'authors on average')

The papers have 4.1985290298819535 authors on average


# Getting all the pickles.
done it assignment6.py
only tested here

In [236]:
def article_author(myroot):
    # Behold my xml tree parser.
    # If it works it aint stupid.

    full_name = []
    pubmed_id = []

    # First loop over every article in the xml file.
    for k in range(0, len(myroot)):
        # Then get the the article tag which is in the medlinecitation
        for child in myroot[k][0]:
            if child.tag == 'Article':
                # Then we need the author list. 
                for second_child in child:
                    if second_child.tag == 'AuthorList':
                        # Then in the authorlist we need all the authors for one paper.
                        for k2 in range(0, len(second_child)):
                            # For every author append the corresponding id to a list
                            pubmed_id.append(myroot[k][0][0].text)
                            # Getting the authors last name
                            lastname = second_child[k2][0].text
                            # Not all authors have their first name listed.
                            try:
                                firstname = second_child[k2][1].text
                            except:
                                firstname = '-'
                            # Create a full name and append it to a list
                            try:
                                name = str(firstname) + ' ' + str(lastname)
                            except:
                                name = 'None'

                            full_name.append(name)

    # Turn both lists created into a dataframe.                                                   
    db = pd.DataFrame((pubmed_id, full_name)).T

    return db

In [240]:
def databaser(files):
    print('working on: ', files)
    mytree = ET.parse(files)
    myroot = mytree.getroot()
    df = article_author(myroot)
    Full_df = Full_df.append(df)

    Full_df.to_pickle(f'/homes/rdalstra/Documents/Programming3/Assignment6/output/{files}.pkl')

In [241]:
def runner(files):
    cpus = 48
    with mp.Pool(cpus) as pool:
        results = pool.map(databaser, files)

In [ ]:
# if __name__ == "__main__":
#     Full_df = pd.DataFrame()
#     path = '/data/dataprocessing/NCBI/PubMed/'
#     files = glob.glob(path + '*.xml')
#     runner(files)